# Reddit wallstreetbets back-test

We use historical wsb subreddit data to back-test a trading strategy. The approach is to use a sliding window approach, for each window, we use OpenAI to perform a summary and suggest trends.

Once we have the trends, we can evaluate the performance of the tickers from the time they first appeared as an investment suggestion. We can test various holding periods, e.g.: 1 day, 1 week, 1 month, etc.

Before you begin, you need to download the wsb data. You can do so by running the following command:

```bash
# install the Google Cloud SDK
brew install google-cloud-sdk

# authenticate using any google account
gcloud auth login

#execute the script
./scripts/extract-wsb-data.sh
```


In [16]:
# reload module

from importlib import reload
import utility

reload(utility)

<module 'utility' from '/Users/tiang/dev/genai/fintechx/notebooks/utility/__init__.py'>

['../data/wsb/export2017.json', '../data/wsb/export2016.json', '../data/wsb/export2015.json', '../data/wsb/export2019.json', '../data/wsb/export2018.json']


<module 'utility' from '/Users/tiang/dev/genai/fintechx/notebooks/utility/__init__.py'>

In [19]:
! pip install pandas

import glob
import os
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import AzureOpenAI
from langchain.embeddings import (
    OpenAIEmbeddings,
    HuggingFaceEmbeddings,
    LlamaCppEmbeddings,
)
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores.qdrant import Qdrant
from langchain.chains.summarize import load_summarize_chain

import qdrant_client

from utility import try_index, try_query

In [21]:
# read in the data
wsb_files = glob.glob("../data/wsb/*.json")
# print(wsb_files)

# file='../data/wsb/export2017.json'
# wsb_df = pd.read_json(file)

wsb_df = pd.concat([pd.read_json(f) for f in wsb_files])
wsb_df

,author,body,created_utc,retrieved_on,subreddit
0,0_-,I am sick of seeing so many wars fought for th...,1509925069,1.512165e+09,wallstreetbets
1,0_o,RemindMe! 3 months,1485566519,1.486498e+09,wallstreetbets
2,1HD,So... Star Wars?,1499010109,1.499844e+09,wallstreetbets
3,1Ra,The upsy-downsy heartbeat. you can expect it t...,1483561320,1.485778e+09,wallstreetbets
4,1jb,Atta boy,1502397647,1.503944e+09,wallstreetbets
...,...,...,...,...,...
1886813,[deleted],[removed],1516927432,1.518287e+09,wallstreetbets
1886814,[deleted],[removed],1515296914,1.517644e+09,wallstreetbets
1886815,[deleted],[removed],1515061012,1.517552e+09,wallstreetbets
1886816,[deleted],[removed],1516761637,1.518222e+09,wallstreetbets


In [22]:
# convert created_utc to datetime
wsb_df["created_utc"] = pd.to_datetime(wsb_df["created_utc"], unit="s")
# convert retrieved_on to datetime
# sort rows by created_utc
wsb_df = wsb_df.sort_values(by="created_utc")
# create a new index based on created_utc and also keep the old column
wsb_df = wsb_df.set_index("created_utc", drop=False)
# convert timestamps to datetimes formatted as strings
wsb_df["retrieved_on"] = pd.to_datetime(wsb_df["retrieved_on"], unit="s").astype(str)
wsb_df["created_utc"] = wsb_df["created_utc"].astype(str)
wsb_df

,author,body,created_utc,retrieved_on,subreddit
created_utc,,,,,
2015-01-01 00:12:14,aspiringanalyst,Complex - yes - but from patient perspective ...,2015-01-01 00:12:14,2015-02-28 11:48:53,wallstreetbets
2015-01-01 03:09:12,[deleted],[deleted],2015-01-01 03:09:12,2015-02-28 11:15:24,wallstreetbets
2015-01-01 06:39:01,phqu88,"nflx, earnings coming up",2015-01-01 06:39:01,2015-02-28 10:43:30,wallstreetbets
2015-01-01 12:41:22,MagnusMcLongcock,Plenty of free apps provide real-time data.,2015-01-01 12:41:22,2015-02-28 10:08:49,wallstreetbets
2015-01-01 17:21:21,proptrader123,"not really, lots of fundemental differences be...",2015-01-01 17:21:21,2015-02-28 09:31:48,wallstreetbets
...,...,...,...,...,...
2019-12-31 23:58:51,omgoptions,Wow thank you for the positive feedback. Its r...,2019-12-31 23:58:51,2020-04-09 16:37:17,wallstreetbets
2019-12-31 23:58:52,[deleted],And that’s enough internet for today,2019-12-31 23:58:52,2020-04-09 16:37:17,wallstreetbets
2019-12-31 23:58:56,[deleted],33ish? Thx Obama.,2019-12-31 23:58:56,2020-04-09 16:37:21,wallstreetbets


In [23]:
# drop any rows that have body == "[removed]" or "[deleted]" or are empty
wsb_df = wsb_df[~wsb_df["body"].isin(["[removed]", "[deleted]", ""])]
wsb_df

,author,body,created_utc,retrieved_on,subreddit
created_utc,,,,,
2015-01-01 00:12:14,aspiringanalyst,Complex - yes - but from patient perspective ...,2015-01-01 00:12:14,2015-02-28 11:48:53,wallstreetbets
2015-01-01 06:39:01,phqu88,"nflx, earnings coming up",2015-01-01 06:39:01,2015-02-28 10:43:30,wallstreetbets
2015-01-01 12:41:22,MagnusMcLongcock,Plenty of free apps provide real-time data.,2015-01-01 12:41:22,2015-02-28 10:08:49,wallstreetbets
2015-01-01 17:21:21,proptrader123,"not really, lots of fundemental differences be...",2015-01-01 17:21:21,2015-02-28 09:31:48,wallstreetbets
2015-01-01 18:55:28,hiphoprising,Yeah I think it'd be better to wait for oil to...,2015-01-01 18:55:28,2015-02-28 09:10:38,wallstreetbets
...,...,...,...,...,...
2019-12-31 23:58:51,omgoptions,Wow thank you for the positive feedback. Its r...,2019-12-31 23:58:51,2020-04-09 16:37:17,wallstreetbets
2019-12-31 23:58:52,[deleted],And that’s enough internet for today,2019-12-31 23:58:52,2020-04-09 16:37:17,wallstreetbets
2019-12-31 23:58:56,[deleted],33ish? Thx Obama.,2019-12-31 23:58:56,2020-04-09 16:37:21,wallstreetbets


In [25]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''
os.environ['AZURE_OPENAI_API_KEY'] = ''
os.environ['OPENAI_API_BASE'] =  'https://fintechx-oai-eus.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [26]:
# get env var
openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]

llm = AzureOpenAI(
    temperature=0.0,
    deployment_name="davinci",
    model="text-davinci-003",
    openai_api_base="https://fintechx-oai-eus.openai.azure.com/",
    openai_api_version="2023-03-15-preview",
    openai_api_key=openai_api_key,
    best_of=3,
)

embeddings = OpenAIEmbeddings(
    deployment="embeddings",
    model="text-embedding-ada-002",
    openai_api_type="azure",
    openai_api_base="https://fintechx-oai-eus.openai.azure.com/",
    openai_api_version="2023-03-15-preview",
    openai_api_key=openai_api_key,
    embedding_ctx_length=8191,
    # for now the chunk_size has to be 1, due to azure openai limitations, Note:
    # We currently do not support batching of embeddings into a single API call.
    # If you receive the error InvalidRequestError: Too many inputs. The max
    # number of inputs is 1. We hope to increase the number of inputs per request
    #  soon., this typically occurs when an array of embeddings is attempted to
    # be passed as a batch rather than a single string. The string can be up to
    # 8191 tokens in length when using the text-embedding-ada-002 (Version 2) model.
    # more: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#embeddings
    # chunk_size=1,
)

WARNING! model is not default parameter.
                    model was transfered to model_kwargs.
                    Please confirm that model is what you intended.
WARNING! openai_api_version is not default parameter.
                    openai_api_version was transfered to model_kwargs.
                    Please confirm that openai_api_version is what you intended.


## (Optional) Override the embeddings

Because Azure OpenAI has very low rate limits, we can use the embeddings from the OpenAI API to speed up the process. To do so, we can override the embeddings with the cell below.

Just append your `OPENAI_API_KEY` to the constructor.


In [ ]:
# embeddings = OpenAIEmbeddings(openai_api_key="sk-***")

In [28]:
# in memory embeddings - heavy on cpu
! pip install sentence_transformers
embeddings = HuggingFaceEmbeddings()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:0031m966.8 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=0bf5acb6f0de0afeeb420ea71fc07829cb4ba241fb98df8dfa5988885a576c27
  Stored in directory: /Users/tiang/Library/Caches/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


/Users/tiang/dev/genai/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## (Optional) Create the index and persist in Qdrant vectorstore

Here, be sure to run `docker compose up -d` in the root of the repo to start the Qdrant server.

The cell below is optional, because it is time consuming, we persist the indexes in the Qdrant running in docker.


In [30]:
# group rows for a week, ending sunday, this way the text will align with the MON market open
weekly_df = wsb_df.resample("W-SUN")
MAX_WORKERS = 3

with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = []
    for _, group in weekly_df:
        # starting timestamp of the group
        start = group.index[0].strftime("%Y-%m-%d")
        # ending timestamp of the group
        end = group.index[-1].strftime("%Y-%m-%d")

        # we've completed indexing up to this date, so skip anything before this
        # conditionally, remove this step if you want to re-index everything
        # if start <= "2016-09-19":
        #     continue

        loader = DataFrameLoader(group, page_content_column="body")
        collection_name = f"wsb_{start}_{end}"

        future = executor.submit(
            try_index,
            collection_name,
            start,
            end,
            VectorstoreIndexCreator(
                vectorstore_cls=Qdrant,
                embedding=embeddings,
                vectorstore_kwargs={
                    "url": "http://localhost:6333",
                    "collection_name": collection_name,
                },
            ).from_loaders,
            [loader],
        )
        # print("submitted:", collection_name)
        futures.append(future)

    for future in as_completed(futures):
        print("finished:", future.result() )

finished: {'start_date': '2015-01-01', 'end_date': '2015-01-04', 'collection_name': 'wsb_2015-01-01_2015-01-04', 'error': 'timed out'}
finished: {'start_date': '2015-01-12', 'end_date': '2015-01-18', 'collection_name': 'wsb_2015-01-12_2015-01-18', 'error': 'timed out'}
finished: {'start_date': '2015-02-02', 'end_date': '2015-02-08', 'collection_name': 'wsb_2015-02-02_2015-02-08', 'error': 'timed out'}
finished: {'start_date': '2015-01-19', 'end_date': '2015-01-25', 'collection_name': 'wsb_2015-01-19_2015-01-25', 'error': 'timed out'}
finished: {'start_date': '2015-02-09', 'end_date': '2015-02-15', 'collection_name': 'wsb_2015-02-09_2015-02-15', 'error': 'timed out'}
finished: {'start_date': '2015-02-16', 'end_date': '2015-02-22', 'collection_name': 'wsb_2015-02-16_2015-02-22', 'error': 'timed out'}
finished: {'start_date': '2015-01-05', 'end_date': '2015-01-11', 'collection_name': 'wsb_2015-01-05_2015-01-11', 'error': 'timed out'}
finished: {'start_date': '2015-01-26', 'end_date': '201

KeyboardInterrupt: 

## Use case 1: Make queries against the collections

Be sure to be running the Qdrant server in docker, and to have the indexes created at least once.

Given any collections that are already in Docker, execute the queries


In [ ]:
# create a Qdrant client and get all of the collections
client = qdrant_client.QdrantClient("http://localhost:6333")
wsb_collections = [
    c.name for c in client.get_collections().collections if c.name.startswith("wsb_")
]

futures = []
with ThreadPoolExecutor(max_workers=2) as executor:
    for collection_name in wsb_collections:
        # print("Found collection:", collection_name)
        # recreate the index
        client = qdrant_client.QdrantClient("http://localhost:6333")
        vectorstore = Qdrant(
            client, collection_name=collection_name, embeddings=embeddings
        )
        restored_index = VectorStoreIndexWrapper(
            vectorstore=vectorstore,
        )
        future = executor.submit(
            try_query,
            collection_name,
            restored_index.query,
            [
                "What are the most trending tickers?",
                "What are the biggest investment opportunities?",
            ],
            llm=llm,
        )
        futures.append(future)
        break
    # wait for all of the futures to complete
    for out in as_completed(futures):
        this_result = out.result()
        answers = this_result["answers"]
        print(this_result["collection_name"])
        for answer in answers:
            print(" ", answer["question"])
            print(" ", answer["answer"])

# get all of the results for further processing..
results = [f.result() for f in futures]

## Use case 2: Summary chain over wsb data

See [lang chain summary docs](https://docs.langchain.com/docs/components/chains/index_related_chains) for the pros and cons on using summary chains.


In [ ]:
# group rows for a week, ending sunday, this way the text will align with the MON market open
weekly_df = wsb_df.resample("W-SUN")

with ProcessPoolExecutor(max_workers=1) as executor:
    futures = []
    for _, group in weekly_df:
        # starting timestamp of the group
        start = group.index[0].strftime("%Y-%m-%d")
        # ending timestamp of the group
        end = group.index[-1].strftime("%Y-%m-%d")
        collection_name = f"wsb_{start}_{end}"

        loader = DataFrameLoader(group, page_content_column="body")
        docs = loader.load()
        print("loaded:", start, end, len(docs))

        chain = load_summarize_chain(llm=llm, chain_type="stuff", verbose=True)

        future = executor.submit(chain.run, docs[:50])

        # print("submitted:", collection_name)
        futures.append(future)
        break

    for future in as_completed(futures):
        print("finished:", future.result())